In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

In [11]:
model_dir = '/home/jihyeparkk/DATA/Data_Mining_Course_Materials/KcELECTRA/checkpoint-300'

relabel_dict = {'가격':0, '디자인':1, '사이즈': 2}
id2label = {val:key for key, val in relabel_dict.items()}
id2label

{0: '가격', 1: '디자인', 2: '사이즈'}

In [6]:
# tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
# tokenizer.save_pretrained(model_dir)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [9]:
def inference(model, input_ids, attention_mask):
    model.eval()
    with torch.no_grad():
        try: outputs = model(input_ids, attention_mask=attention_mask, return_dict=True)
        except: return None
    logits = outputs['logits']
    predicted_label = torch.argmax(logits)
    return predicted_label.item()

def encode_for_inference(device, tokenizer, text):
    data = {key: torch.tensor(val) for key, val in tokenizer(text, truncation=True, padding='max_length', max_length=512).items()}
    return torch.unsqueeze(data['input_ids'], 0).to(device), torch.unsqueeze(data['attention_mask'], 0).to(device)

In [12]:
texts = ["이 마스크는 저렴해서 좋아요", "이 마스크는 예뻐서 좋아요", "이 마스크는 크기가 커서 좋아요"]
for text in texts:
    input_ids, attention_mask = encode_for_inference(device, tokenizer, text)
    print('{} -> Classified as "{}"" aspect.'.format(text, id2label[inference(model, input_ids, attention_mask)]))

이 마스크는 저렴해서 좋아요 -> Classified as "가격"" aspect.
이 마스크는 예뻐서 좋아요 -> Classified as "디자인"" aspect.
이 마스크는 크기가 커서 좋아요 -> Classified as "사이즈"" aspect.
